In [ ]:
!pip install tensorflow
!pip install tensorflow-probability

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

# Load data
df = pd.read_csv('../../data/df_no_outliers.csv')

columns_to_drop = ['Station_Latitude', 'Station_Longitude']
df = df.drop(columns_to_drop, axis=1)

# Define features and target
features = df.drop(['AttendanceTimeSeconds'], axis=1)
target = df['AttendanceTimeSeconds']

# Define categorical and numerical features
categorical_cols = features.select_dtypes(exclude=np.number).columns.tolist()
numerical_cols = features.select_dtypes(include=np.number).columns.tolist()

# Use ColumnTransformer to apply different preprocessing to numerical and categorical features
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Fit the preprocessor and transform the data
X_train = preprocessor.fit_transform(X_train).toarray()
X_test = preprocessor.transform(X_test).toarray()

# Model 1: More layers, 'relu' activation
def build_dnn_model_1():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1)),
    ])
    return model

# Model 2: More neurons, 'tanh' activation
def build_dnn_model_2():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1)),
    ])
    return model

# Model 3: Different combination - more layers and neurons, 'sigmoid' activation
def build_dnn_model_3():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='sigmoid'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1)),
    ])
    return model

def build_dnn_model_4():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1))
    ])
    return model

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# List of models
models = [build_dnn_model_1, build_dnn_model_2, build_dnn_model_3, build_dnn_model_4]

# Loop through models
for i, model_func in enumerate(models):
    # Create the model
    model = model_func()

    # Compile the model
    model.compile(optimizer=tf.optimizers.Adam(), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the predictions
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Model {i+1}: Mean Absolute Error: {mae}, Mean Squared Error: {mse}")

    errors = y_test - y_pred.reshape(-1)

    # Display instances with highest error
    print("Instances with largest error:")
    print(errors.abs().sort_values(ascending=False).head(10))


Epoch 1/50
 6862/13680 [==============>...............] - ETA: 20s - loss: nan

KeyboardInterrupt: 